### Analise dos tempos de performance entre diferentes tolerancias

In [17]:

import pandas as pd
import os 
from quantfin.strategy import Strategy 
from quantfin.datawrangling import save_to_pickle, return_from_pickle

returns, b_sp500, b_tyde = return_from_pickle(path = os.path.join('dataset',"processed", "data_input.pickle") )


returns = pd.read_csv(os.path.join('dataset',"processed", 'returns.csv'))
returns.set_index('Unnamed: 0', drop = True, inplace = True) 

tolerance_list = [None, 0.01, 0.001]
max_w_list = [0.05]
time_os_list = [3]

count = 22
for tolerance in tolerance_list:
    for max_w in max_w_list:
        for time_os in time_os_list:
            portfolio = Strategy(method = 'max_sharpe',
                            name = 'Max Sharpe {}'.format(str(count)),
                            log_path = 'logs/max_sharpe{}.csv'.format(str(count)),
                            rets = returns,
                            time_is = 12, 
                            time_os = time_os, 
                            max_w= max_w,
                            date_from = '2013-01-01', 
                            date_to = '2018-11-30',
                            threads= 12,
                            minimization_tolerance= tolerance)
            save_to_pickle('portfolios/max_sharpe{}.pickle'.format(str(count)), portfolio)
            count = count + 1 

[2019-12-27 10:57:03] Starting Defining Variables.
[2019-12-27 10:57:03] Finished Defining Variables.
[2019-12-27 10:57:03] Starting Generating dates and filtering returns.
[2019-12-27 10:57:03] Finished Generating dates and filtering returns.
[2019-12-27 10:57:03] Starting Calculating weights from 20 periods
[2019-12-27 10:57:04] Starting Calculating portfolio weights from 2013-01-01 to 2013-12-31
[2019-12-27 10:57:04] Starting Calculating portfolio weights from 2013-04-01 to 2014-03-31
[2019-12-27 10:57:04] Starting Calculating portfolio weights from 2013-07-01 to 2014-06-30
[2019-12-27 10:57:04] Starting Calculating portfolio weights from 2013-10-01 to 2014-09-30
[2019-12-27 10:57:05] Starting Calculating portfolio weights from 2014-01-01 to 2014-12-31
[2019-12-27 10:57:05] Starting Calculating portfolio weights from 2014-04-01 to 2015-03-31
[2019-12-27 10:57:06] Starting Calculating portfolio weights from 2014-07-01 to 2015-06-30
[2019-12-27 10:57:06] Starting Calculating portfolio

In [21]:

import pandas as pd 
from quantfin.datawrangling import return_from_pickle
from quantfin.analysis import compare_strategies
import os 
returns, b_sp500, b_tyde = return_from_pickle(path = os.path.join('dataset',"processed", "data_input.pickle") )
tolerance_list = ['1e-6', '0.01', '0.001']

index_list = [20, 21, 22]
portfolio_list = []
diff_list = []
for n,i in enumerate(index_list):
    data = pd.read_csv('logs/max_sharpe{}.csv'.format(i), sep = ';')
    data.sort_values(by=['id'])
    data['date'] = pd.to_datetime(data['date'])
    diff =  data['date'].iloc[-1] - data['date'][0] 
    diff_list.append
    print('Portfolio', str(i) + ':',diff)
    portfolio = return_from_pickle('portfolios/max_sharpe{}.pickle'.format(i))
    portfolio_list.append(portfolio)
    
compare_strategies(portfolio_list,b_sp500 )



Portfolio 20: 0 days 01:37:25
Portfolio 21: 0 days 00:22:32
Portfolio 22: 0 days 00:33:42


,Annaul Return,Annual Volatility,Beta,Sharpe,Sortino,Treynor,Starr,Kurtosis,Skew,CVar 5%,Var 5%,Maximum Drawdown,Avg. Turnover:,avg. Size:
Strategy,,,,,,,,,,,,,,
Max Sharpe 20,16.23%,15.63%,0.99,1.04,0.08,0.16,-672.05%,1.74,-0.63,-2.41%,-1.67%,20.05%,58.81%,27
Max Sharpe 21,16.28%,15.61%,0.99,1.04,0.08,0.16,-675.62%,1.71,-0.62,-2.41%,-1.63%,19.68%,58.84%,28
Max Sharpe 22,16.33%,15.63%,1.00,1.05,0.08,0.16,-676.60%,1.75,-0.63,-2.41%,-1.66%,19.92%,58.60%,29
Tyde,15.15%,16.48%,1.16,0.92,0.07,0.13,-573.31%,2.28,-0.66,-2.64%,-1.82%,18.31%,0.00%,0
SP 500,8.38%,12.80%,1.00,0.65,0.05,0.08,-412.18%,3.20,-0.54,-2.03%,-1.36%,14.16%,0.00%,0


In [ ]:
seconds_list = [x.total_seconds()/60 for x in diff_list]
seconds_list.sort(reverse = True)

print(seconds_list)

import plotly.graph_objects as go
colors = ['lightslategray',] * 3

colors[2] = '#007580'
data = [go.Bar(y=seconds_list, x=['1e-6', '0.001', '0.01'], marker_color = colors)]
layout = go.Layout(xaxis=dict(type='category'))
fig = go.Figure(data = data, layout = layout)
fig.update_layout(
    title='Execution time',
    xaxis_title= 'Tolerance',
    yaxis_title = 'Minutes',
    )
fig.show()